### Define configuration files

In [ ]:
from box.sim_box import SimulationBox
from potential.force import Force
from cell.cell import Cell
from substrate.substrates import Substrate
from helper_functions import helper_functions as hf
from visuals.figure import Figure
from polarity import polarity

import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
import os
import time
from importlib import reload

reload(polarity)

### Some random functions

In [ ]:
def _masked(arr, masker):
    arr_masked = np.ones(arr.shape) * np.nan
    i, j = np.where(masker >= 0.5)
    arr_masked[i, j] = arr[i, j]
    return arr_masked


def plot_probs(cell, p1, p2, p, c, time, p_ratio):
    cntr = cell.contour[0][:, ::-1]

    if not isinstance(p1, np.ndarray):
        p1 = np.array([p1] * cntr.shape[0])

    if not isinstance(p2, np.ndarray):
        p2 = np.array([p2] * cntr.shape[0])

    plt.figure(figsize=(6, 6))
    plt.subplot(221)
    plt.title("filopodia")
    plt.scatter(cntr[:, 0], cntr[:, 1], c=p1, cmap="coolwarm")
    plt.colorbar()

    plt.subplot(222)
    plt.title("feedback")
    plt.scatter(cntr[:, 0], cntr[:, 1], c=p2, cmap="coolwarm")
    plt.colorbar()
    plt.scatter(*c, color="black", s=20)

    plt.subplot(223)
    plt.title("total")
    plt.scatter(cntr[:, 0], cntr[:, 1], c=p, cmap="coolwarm")
    plt.colorbar()

    plt.subplot(224)
    plt.title("Tension")
    plt.plot(time, p_ratio)
    plt.xlabel("Time")
    plt.ylabel(r"$P/P_0$")

    plt.subplots_adjust(wspace=1, hspace=0.7)
    plt.show()

### Initialize system

In [ ]:
def _build_system(simbox, cell_config, cell_rng_seed):
    # unpack
    N_mesh, L_box = simbox.N_mesh, simbox.L_box

    # define base substrate
    sub_config = simbox.sub_config
    sub_dim = simbox.sub_config["sub_sep"]
    xi = sub_config["xi"]
    kind = sub_config["kind"]
    # buffer = sub_config["buffer"]
    sub = Substrate(N_mesh, L_box, xi)
    if kind == "two-state":
        chi = sub.two_state_sub(bridge_width=17, delta_centers=sub_dim)
    elif kind == "rectangular":
        chi = sub.rectangular()
    else:
        raise ValueError(f"{kind} for substrate is not understood.")

    # initialize cells
    # set the cumulative substrate they will interact with
    cell = Cell(cell_config[0], simbox, cell_rng_seed)
    cell.W = 0.5 * cell.g * chi

    return cell, chi


def _define_paths(gid):
    SIMBOX_CONFIG = f"../configs/IM/grid_id{gid}/simbox.yaml"
    ENERGY_CONFIG = "../configs/energy.yaml"

    CELL_CONFIG = [f"../configs/IM/grid_id{gid}/cell.yaml"]

    return dict(
        simbox=SIMBOX_CONFIG,
        energy=ENERGY_CONFIG,
        cell=CELL_CONFIG,
    )


def _mvg_generator(cell):
    import numpy as np
    from polarity.mvgaussian import MVGaussian

    N_mesh = cell.simbox.N_mesh
    d = np.linspace(0, N_mesh, N_mesh)
    x, y = np.meshgrid(d, d)
    X = np.array(list(zip(x.flatten(), y.flatten())))

    cell.mvg_gen = MVGaussian(X)

In [ ]:
# define various paths
gid = 21
paths = _define_paths(gid)

# initialize the simulation box
simbox = SimulationBox(paths["simbox"])
seed = int(time.time())
cell, chi = _build_system(simbox, paths["cell"], cell_rng_seed=seed)

# initialize the force calculator
force_calculator = Force(paths["energy"])

# noise patch -- init MVG generator for the cell
_mvg_generator(cell)

###  🪛  Hyperparameters

In [ ]:
N = 1000
cap = 250

### Evolve the cell

In [ ]:
from scipy.interpolate import RegularGridInterpolator


def prob(cell, grad_phi, field, method):
    cntr = cell.contour[0]
    grad_phi_norm = np.sqrt(np.sum(grad_phi * grad_phi, axis=0)) + 1e-10
    n_hat = -grad_phi / grad_phi_norm
    l = 0
    cntr = np.array(
        [
            [y - l * n_hat[1][int(y), int(x)], x - l * n_hat[0][int(y), int(x)]]
            for y, x in cntr
        ]
    )

    p_field = np.where(field < 0, 1e-5, field)
    x = np.arange(cell.simbox.N_mesh)
    interp = RegularGridInterpolator((x, x), p_field, method=method)

    cntr = cell.contour[0][:, ::-1]
    in_frame_cntr = cntr * cell.simbox.dx - cell.cm[1]
    radii = np.linalg.norm(in_frame_cntr, axis=1)
    return interp(cntr)


def poisson_add_time(rate):
    tau_add = cell.rng.poisson(rate)
    while tau_add == 0:
        tau_add = cell.rng.poisson(rate)
    return tau_add


def poisson_patch_mag(rate, positive_only=True):
    mag = cell.rng.poisson(rate)
    return cell.rng.poisson(rate) if positive_only else mag * cell.rng.randint(-1, 2)


def gaussian_patch_mag(mean, std):
    return cell.rng.normal(mean, std)

In [ ]:
from IPython import display
import os

t = []
p_ratio = []
save_path = f"../output/IM/grid_id{gid}/run_0/visuals/"

if not os.path.exists(save_path):
    os.makedirs(save_path)

for n in range(N):
    grad_x, grad_y, _ = hf.compute_gradients(cell.phi, cell.simbox.dx)
    grad_phi = np.array([grad_x, grad_y])
    eta = cell.eta
    phi = cell.phi
    dt = cell.simbox.dt

    # contour PMF to add mvg patch
    p1 = polarity.cntr_probs_filopodia(cell, grad_phi, chi, delta_l=5)
    p2 = polarity.cntr_probs_feedback(cell, grad_phi)
    cntr_probs = p1 * p2
    cntr_probs /= cntr_probs.sum()

    # add MVG patch according to a Poisson process
    tau_add = hf._poisson_add_time(cell.rng, cell.pol_model_kwargs["add_rate"])
    if n % tau_add == 0:
        mag = cell.rng.normal(
            loc=cell.pol_model_kwargs["mag_mean"],
            scale=cell.pol_model_kwargs["mag_std"],
        )
        mvg_patch = mag * polarity.mvg_patch(cell, cntr_probs)
    else:
        mvg_patch = 0

    # phi_(n+1)
    phi_i_next, dF_dphi = hf._update_field(cell, grad_phi, force_calculator)

    if n % cap == 0:
        display.clear_output(wait=True)
        t.append(n)
        cntr = cell.contour[0][:, ::-1] * cell.simbox.dx
        perim = np.sqrt(np.sum(np.diff(cntr, axis=0) ** 2, axis=1)).sum()
        p_ratio.append(perim / (2 * np.pi * 4.5))

        plot_probs(cell, p1, p2, cntr_probs, cell.contour[0][0][::-1], t, p_ratio)

        Figure.view_pol_field(
            cell,
            chi,
            dpi=150,
            path=save_path + f"img_{n//cap}.png",
        )

    # polarization field (n+1)
    p_field_next = polarity.update_field(cell, chi, mvg_patch, cell.pol_model_kwargs)

    # compute motility forces at time n
    fx_motil, fy_motil = force_calculator.cyto_motility_force(cell, grad_phi, chi)

    # compute thermodynamic forces at time n
    fx_thermo = dF_dphi * grad_x
    fy_thermo = dF_dphi * grad_y

    # UPDATE class variables now
    cell.phi = phi_i_next
    cell.p_field = p_field_next
    cell.contour = hf.find_contour(cell.phi)
    cell.cm = hf.compute_CM(cell)
    cell.v_cm = hf.compute_v_CM(cell)
    cell.vx = (fx_thermo + fx_motil) / eta
    cell.vy = (fy_thermo + fy_motil) / eta


---

### Debugging

In [ ]:
def interpolate(xy_array, dx=0.001):
    # pts.shape == (N, 2) with (x, y) format

    def _interpolate_segment(pts):
        xmin = pts[:, 0].min()
        xmax = pts[:, 0].max()
        ymin = pts[:, 1].min()
        ymax = pts[:, 1].max()
        d = np.sqrt((xmax - xmin) ** 2 + (ymax - ymin) ** 2)
        resolution = int(d / dx)

        if np.fabs(xmax - xmin) < 1e-5:
            X = np.asarray([xmin] * resolution)
            Y = np.linspace(ymin, ymax, resolution)
            return np.hstack([X.reshape(-1, 1), Y.reshape(-1, 1)])

        X = np.linspace(xmin, xmax, resolution)
        return np.hstack(
            [X.reshape(-1, 1), np.interp(X, pts[:, 0], pts[:, 1]).reshape(-1, 1)]
        )

    return np.concatenate(
        [
            _interpolate_segment(
                xy_array[i : i + 2][np.argsort(xy_array[i : i + 2][:, 0])]
            )
            for i in range(xy_array.shape[0] - 1)
        ],
        axis=0,
    )

In [ ]:
from skimage import measure


phi = cell.phi
im_kwargs = {"origin": "lower", "extent": [0, 50, 0, 50]}

sk_cntr = cell.contour[0][:, ::-1] * cell.simbox.dx
# cntr = plt.contour(phi, levels=[0.5], colors=["black"], linewidths=[2], **im_kwargs)
xy_cntr = cntr.collections[0].get_paths()[0].vertices

from shapely import linestrings, LineString

line = LineString(xy_cntr)
diffs = np.fabs(np.diff(xy_cntr, axis=0)[:, 0])
d_min = diffs[diffs > 1e-3].min()
print(d_min)
N = int(line.length / d_min)
pts = [line.interpolate(d_min * i) for i in range(N)]
pts = np.array([[pt.x, pt.y] for pt in pts])
pts = np.unique(pts, axis=0)

plt.scatter(xy_cntr[0, 0], xy_cntr[0, 1], color="red", s=30)
plt.scatter(pts[:, 0], pts[:, 1], color="green", s=30, alpha=0.05)
plt.ylim((22, 28))
plt.xlim((22, 28))
plt.axis("equal")


120,000 step run is sufficient to see asymmetry based on `corners_only_2`

[ ] Submit small runs for gid 21, maybe 96 at 120,000:
   - Collect stats about selected contour pts
        - Position
        - Magnitude of patch

   + Collect stats about # of contour pts to the left and right of CM

[ ] Rerun with contour points uniformly interpolated

# Debugging asymmetry from single_run

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


res_df = pd.read_csv("../output/IM/grid_id21/run_0/result.csv")

res_df.x *= 6
res_df.y *= 6

plt.figure(figsize=(3, 2))
plt.hist(res_df.x, bins=32, density=True)
plt.show()

In [ ]:
asym_df = pd.read_csv("../output/IM/grid_id21/run_0/asym.csv").query("cs_x > -1")
asym_df.x *= 6
asym_df.y *= 6
asym_df.cs_x *= 6
asym_df.cs_y *= 6

cm = asym_df[["x", "y"]].values
picked_c = asym_df[["cs_x", "cs_y"]].values

ratio = picked_c[:, 0] - cm[:, 0]

plt.figure(figsize=(3, 2))
plt.hist(ratio, bins=32, density=True)
plt.show()